Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name below:

In [ ]:
let name = "Pranav Marepalli"
let rollno = "CS23B034"

## Important notes about grading:

1. **Compiler errors:** All code you submit must compile. Programs that do not compile will probably receive an automatic zero. If you are having trouble getting your assignment to compile, please visit consulting hours. If you run out of time, it is better to comment out the parts that do not compile, than hand in a more complete file that does not compile.
2. **Late assignments:** Please carefully review the course website's policy on late assignments, as all assignments handed in after the deadline will be considered late. Verify on moodle that you have submitted the correct version, before the deadline. Submitting the incorrect version before the deadline and realizing that you have done so after the deadline will be counted as a late submission.

# Lambda Calculus Interpreter

In this assignment, you will implement lambda calculus interpreters that use different reduction strategies. The abstract syntax tree (AST) for lambda expressions is the one that we have seen in class:


```ocaml
type expr = 
  | Var of string
  | Lam of string * expr
  | App of expr * expr
```


You are provided a parser function `parse_string` that converts a string to this AST and a printer function `string_of_expr` that converts the AST to string. For example, 

In [ ]:
#use "init.ml"
open Syntax
let parse_string = Lambda_parse.parse_string
let string_of_expr = string_of_expr

let _ = parse_string "(\\x.x) (\\y.y)"
let _ = string_of_expr (App (Var "x",Lam("y",App(Var "y", Var "x"))))

You will need some helper functions to operate over sets. Since we have not studied set data structure in OCaml. We will use lists instead and implement set functionality on top of lists. You can use the functions from the OCaml list standard library for this assignment. The documentation for OCaml list module is available in the [OCaml manual](https://caml.inria.fr/pub/docs/manual-ocaml/libref/List.html). 

## Problem 1

Implement a function

```ocaml
mem : 'a -> 'a list -> bool
```

`mem e l` returns `true` if the element `e` is present in the list. Otherwise, it returns false. 

In [ ]:
let mem e l = 
  (* YOUR CODE HERE *)
  List.exists (fun x -> x = e) l
  (* raise (Failure "Not implemented") *)

In [ ]:
(* 5 points *)
assert (mem "b" ["a";"b";"c"] = true);
assert (mem "x" ["a";"b";"c"] = false)

## Problem 2

Implement a function

```ocaml
remove : 'a -> 'a list -> 'a list
```

`remove e l` returns a list `l'` with all the element in `l` except `e`. `remove` also preserves the order of the elements not removed. If `e` is not present in `l`, then return `l`.

In [ ]:
let remove e l = 
  (* YOUR CODE HERE *)
  List.filter (fun x -> x <> e) l
  (* raise (Failure "Not implemented") *)

In [ ]:
(* 5 points *)
assert (remove "b" ["a";"b";"c"] = ["a";"c"]);
assert (remove "x" ["a";"b";"c"] = ["a";"b";"c"])

## Problem 3

Implement a function

```ocaml
union : string list -> string list -> string list
```

`union l1 l2` performs set union of elements in `l1` and `l2`. The elements in the result list `l` must be lexicographically sorted. Hint: You may want to use the functions `List.sort` and `remove_stutter` from assignment 1 to implement union. Here is an example of using `List.sort`. 

In [ ]:
assert (List.sort String.compare ["x";"a";"b";"m"] = ["a";"b";"m";"x"])

In [ ]:
let union l1 l2 = 
  (* YOUR CODE HERE *)
  List.sort_uniq String.compare (l1 @ l2)
  (* raise (Failure "Not implemented") *)

In [ ]:
(* 5 points *)
assert (union ["a"; "c"; "b"] ["d"; "b"; "x"; "a"] = ["a"; "b"; "c"; "d"; "x"])

## Problem 4

Implement a function

```ocaml
add : 'a -> 'a list -> 'a list
```

`add e l` does a set addition of element `e` to list `l` and returns a list. The resultant list is sorted. 

In [ ]:
let add e l = 
  (* YOUR CODE HERE *)
  let l1 = if mem e l then l else e::l in
  List.sort compare l1
  (* raise (Failure "Not implemented") *)

In [ ]:
(* 5 points *)
assert (add "b" ["a";"c"] = ["a";"b";"c"]);
assert (add "a" ["c"; "a"] = ["a";"c"])

## Substitution

At the heart of reducing lambda expressions is substitution. Recall from the lecture that substitution requires us to generate fresh variable names that is different from every other name used in the current context. We will use the following helper function to generate fresh names.

In [ ]:
let r = ref 0

let fresh s =                                                                       
  let v = !r in                                                                     
  r := !r + 1;                                                                      
  s ^ (string_of_int v)

It uses mutability features of OCaml which we will study in later lectures. You can use the `fresh` function as follows:

In [ ]:
let a = fresh "a"
let b = fresh "b"

## Problem 5

Implement a function 

```ocaml
free_variables: expr -> string list
```

that returns the free variables in the given lambda term.

In [ ]:
let rec free_variables e = 
  (* YOUR CODE HERE *)
  match e with
  | Var x -> [x]
  | Lam (x, body) -> remove x (free_variables body)
  | App (e1, e2) -> (free_variables e1) @ (free_variables e2)
  (* raise (Failure "Not implemented") *)

In [ ]:
(* 20 points *)
assert (free_variables (parse_string "\\x.x") = []);
assert (free_variables (parse_string "\\x.y") = ["y"])

## Problem 6

Implement the function

```ocaml
substitute : expr -> string -> expr -> expr
```

where `substitute e x v` does `e[v/x]`. For renaming `x` in `Lam(x,y)` with a fresh name, use `Lam (fresh x, ...)`.

In [ ]:
let rec substitute expr a b =
  (* YOUR CODE HERE *)
  match expr with
  | Var x -> if x = a then b else Var x
  | Lam (x, body) -> 
      if x = a then Lam (x, body)
      else if mem x (free_variables b) then
        let x' = fresh x in
        Lam (x', substitute (substitute body x (Var x')) a b)
      else
        Lam (x, substitute body a b)
  | App (e1, e2) -> App (substitute e1 a b, substitute e2 a b)
  (* raise (Failure "Not implemented") *)

In [ ]:
(* 20 points *)
assert (alpha_equiv 
          (substitute (parse_string "\\y.x") "x" (parse_string "\\z.z w")) 
          (parse_string "λy.λz.z w"));
assert (alpha_equiv 
          (substitute (parse_string "\\x.x") "x" (parse_string "y"))
          (parse_string "λx.x"));
assert (alpha_equiv 
          (substitute (parse_string "\\x.y") "y" (parse_string "x"))
          (parse_string "λx0.x"))

## Problem 7

Implement a single step of the call-by-value reduction. Implement the function

```ocaml
reduce_cbv : expr -> expr option
```

which does a single step of the call-by-value reduction. Recall that call-by-value reduction is determinisitic. Hence, if reduction is possible, then a single rule applies. `reduce e` returns `Some e'` if reduction is possible and `e'` is the new expression. `reduce e` returns `None` if reduction is not possible.

In [ ]:
let rec reduce_cbv e =
  (* YOUR CODE HERE *)
  match e with
  | App (Lam (x, body), v) when 
    (match v with Lam _ | Var _ -> true | App _ -> false) ->
    Some (substitute body x v)
  | App (e1, e2) ->
    (match reduce_cbv e1 with
    | Some e1' -> Some (App (e1', e2))
    | None ->
      (match reduce_cbv e2 with
      | Some e2' -> Some (App (e1, e2'))
      | None -> None
      )
    )
  | _ -> None
  (* raise (Failure "Not implemented") *)

In [ ]:
(* 20 points *)
begin match reduce_cbv (parse_string "(\\x.x) ((\\x.x) (\\z.(\\x.x) z))") with
| Some expr -> assert (alpha_equiv expr (parse_string "(λx.x) (λz.(λx.x) z)"))
| None -> assert false
end;
        
begin match reduce_cbv (parse_string "(λx.x) (λz.(λx.x) z)") with
| Some expr -> assert (alpha_equiv expr (parse_string "λz.(λx.x) z"))
| None -> assert false
end;
        
assert (reduce_cbv (parse_string "λz.(λx.x) z") = None);
        
begin match reduce_cbv (parse_string "(λx.y) ((λx.x x) (λx.x x))") with
| Some expr -> assert (alpha_equiv expr (parse_string "(λx.y) ((λx.x x) (λx.x x))"))
| None -> assert false
end;

assert (reduce_cbv (parse_string "x y z") = None)


## Problem 8

Implement a single step of the call-by-name reduction. Implement the function

```ocaml
reduce_cbn : expr -> expr option
```

The rest of the instructions are same as `reduce_cbv`.

In [ ]:
let rec reduce_cbn e =
  (* YOUR CODE HERE *)
  match e with
  | App (Lam (x, body), arg) ->
    Some (substitute body x arg)
  | App (e1, e2) ->
    (match reduce_cbn e1 with
    | Some e1' -> Some (App (e1', e2))
    | None -> None
    )
  | _ -> None
  (* raise (Failure "Not implemented") *)

In [ ]:
(* 20 points *)
begin match reduce_cbn (parse_string "(\\x.x) ((\\x.x) (\\z.(\\x.x) z))") with
| Some expr -> assert (alpha_equiv expr (parse_string "(λx.x) (λz.(λx.x) z)"))
| None -> assert false
end;
        
begin match reduce_cbn (parse_string "(λx.x) (λz.(λx.x) z)") with
| Some expr -> assert (alpha_equiv expr (parse_string "λz.(λx.x) z"))
| None -> assert false
end;
        
assert (reduce_cbn (parse_string "λz.(λx.x) z") = None);
        
begin match reduce_cbn (parse_string "(λx.y) ((λx.x x) (λx.x x))") with
| Some expr -> assert (alpha_equiv expr (parse_string "y"))
| None -> assert false
end;

begin match reduce_cbn (parse_string "(\\x.x x) ((\\z.z) y)") with
| Some expr -> assert (alpha_equiv expr (parse_string "(λz.z) y ((λz.z) y)"))
| None -> assert false
end;

assert (reduce_cbn (parse_string "x y z") = None)


## Problem 9

Implement a single step of the normal order reduction. Implement the function

```ocaml
reduce_normal : expr -> expr option
```

The rest of the instructions are same as `reduce_cbv`.

In [ ]:
let rec reduce_normal e =
  (* YOUR CODE HERE *)
  match e with
  | App (Lam (x, body), arg) ->
    Some (substitute body x arg)
  | App (e1, e2) ->
    (match reduce_normal e1 with
    | Some e1' -> Some (App (e1', e2))
    | None ->
      (match reduce_normal e2 with
      | Some e2' -> Some (App (e1, e2'))
      | None -> None
      )
    )
  | Lam (x, body) ->
    (match reduce_normal body with
    | Some body' -> Some (Lam (x, body'))
    | None -> None
    )
  | _ -> None
  (* raise (Failure "Not implemented") *)

In [ ]:
(* 20 points *)
begin match reduce_normal (parse_string "(\\x.x) ((\\x.x) (\\z.(\\x.x) z))") with
| Some expr -> assert (alpha_equiv expr (parse_string "(λx.x) (λz.(λx.x) z)"))
| None -> assert false
end;
        
begin match reduce_normal (parse_string "(λx.x) (λz.(λx.x) z)") with
| Some expr -> assert (alpha_equiv expr (parse_string "λz.(λx.x) z"))
| None -> assert false
end;
        
begin match reduce_normal (parse_string "λz.(λx.x) z") with
| Some expr -> assert (alpha_equiv expr (parse_string "λz. z"))
| None -> assert false
end;
        
begin match reduce_normal (parse_string "(λx.y) ((λx.x x) (λx.x x))") with
| Some expr -> assert (alpha_equiv expr (parse_string "y"))
| None -> assert false
end;

begin match reduce_normal (parse_string "(\\x.x x) ((\\z.z) y)") with
| Some expr -> assert (alpha_equiv expr (parse_string "(λz.z) y ((λz.z) y)"))
| None -> assert false
end;
        
begin match reduce_normal (parse_string "f (\\x.x x) ((\\z.z) y)") with
| Some expr -> assert (alpha_equiv expr (parse_string "f (λx.x x) y"))
| None -> assert false
end;

begin match reduce_normal (parse_string "(\\x.(\\z.z) y) (\\x.x x)") with
| Some expr -> assert (alpha_equiv expr (parse_string "(λz.z) y"))
| None -> assert false
end


In [ ]:
let rec eval log depth reduce expr =                     
  if depth = 0 then failwith "non-termination?"                                  
  else                                                                     
    begin match reduce expr with
    | None -> expr
    | Some expr' ->
      if log then print_endline ("= " ^ (string_of_expr expr'));                 
      eval log (depth-1) reduce expr'                                    
    end
  
let eval_cbv = eval true 1000 reduce_cbv
let eval_cbn = eval true 1000 reduce_cbn
let eval_normal = eval true 1000 reduce_normal

In [ ]:
(* 10 points *)
let zero = parse_string "\\f.\\x. x" in
let one = parse_string "\\f.\\x. f x" in
let two = parse_string "\\f.\\x. f (f x)" in
let three = parse_string "\\f.\\x. f (f (f x))" in

let plus = parse_string "λm. λn. λs. λz. m s (n s z)" in
let mult = parse_string "λm. λn. λs. λz. m (n s) z" in

assert (alpha_equiv (eval_normal (App (App (plus, one), two))) three);
print_endline "";
assert (alpha_equiv (eval_normal (App (App (mult, one), three))) three);
print_endline "";
assert (alpha_equiv (eval_normal (App (App (mult, zero), three))) zero)